# leda demo: plotly

In [ ]:
import abc
import dataclasses
from typing import Optional

import numpy as np
import pandas as pd
import plotly.graph_objs as go
from typing_extensions import override

import leda

In [ ]:
leda.init("plotly")

## Info

**Widgets**

Use the `%%interact expr0;expr1;...` cell magic to set widgets for that cell.

Each expression is of the form `x=y`, where `x` becomes the local var of the cell and `y` can be a:
- `list` to indicate choices for a dropdown widget
- `tuple` to indicate values for an int slider (start, stop, and optional step).

E.g.:

```python
%%interact column=list(df.columns)
%%interact column=list(df.columns);mult=[1, 2, 3]
%%interact column=list(df.columns);window=(10, 50)
%%interact column=list(df.columns);window=(10, 50, 5)
```

**Table of Contents**

Use the `%toc` line magic to substitute with a table of contents in static mode.

**Toggles**

Click the `Toggle input cells` button at the bottom to reveal input cells.

## Data

Using randomly generated data (with fixed seed).

In [ ]:
df = pd.DataFrame(
    np.random.RandomState(42).rand(100, 10), columns=list("abcdefghij")
)

## Visualization

In [ ]:
def plot_df(df: pd.DataFrame, title: Optional[str] = None) -> go.Figure:
    fig = go.Figure()

    colors = [
        "rgb(0, 0, 255)",
        "rgb(255, 0, 0)",
        "rgb(0, 255, 0)",
        "rgb(0, 0, 0)",
        "rgb(128, 128, 128)",
        "rgb(255, 255, 0)",
        "rgb(0, 255, 255)",
        "rgb(255, 0, 255)",
        "rgb(200, 200, 200)",
    ]
    for idx, column in enumerate(df.columns):
        color = colors[idx % len(colors)]

        fig.add_trace(
            go.Scatter(
                x=df.index,
                y=df[column],
                mode="lines",
                marker={"color": color},
                name=str(column),
            )
        )

    if title:
        fig.update_layout(title=title)

    return fig

### Simple

In [ ]:
%%interact column=list(df.columns);mult=[1, 2, 3]
plot_df(df[[column]] * mult, title=f"column={column!r}, mult={mult}")

In [ ]:
%%interact column=list(df.columns);window=(10, 50, 5)
fig = plot_df(
    df[[column]].iloc[-window:], title=f"column={column!r}, window={window}"
)
fig

### Objects as Params

In [ ]:
@dataclasses.dataclass(frozen=True)
class Calculator(abc.ABC):
    @abc.abstractmethod
    def calc(self, df: pd.DataFrame) -> pd.DataFrame: ...


@dataclasses.dataclass(frozen=True)
class CumSumCalculator(Calculator):
    @override
    def calc(self, df: pd.DataFrame) -> pd.DataFrame:
        return df.cumsum()


@dataclasses.dataclass(frozen=True)
class EWMMeanCalculator(Calculator):
    com: float

    @override
    def calc(self, df: pd.DataFrame) -> pd.DataFrame:
        return df.ewm(com=self.com).mean()


calcs = [
    CumSumCalculator(),
    EWMMeanCalculator(com=5),
    EWMMeanCalculator(com=10),
]

In [ ]:
%%interact column_group=["abc", "def", "ghij"];calc=calcs
calced_df = calc.calc(df[list(column_group)])
title = f"column_group={column_group!r}, calc={calc}"
plot_df(calced_df, title=title)